# Estimating Pressure Given Total Masses and Temperature

The goal of this notebook is to find an efficient and reasonably accurate method to estimate the pressure expected in the Parr reactor given a fixed mass of polyol, cyclopentane, and carbon dioxide at a certain temperature T.

In [150]:
import numpy as np
import pandas as pd

import glob
import os

# SET PARAMETERS
data_folder = '..\\..\\..\\Wang\\PC-SAFT\\Data\\'
data_file_format = 'ppg1k_T310_P*_phase_data.csv' # set temperature to desired value
# pressure
p_min = 3 # minimum pressure considered in MPa
p_max = 7 # maximum pressure considered in MPa
# parr
V = 1200 # total volume of Parr reactor [mL]
m_co2 = 202 # mass of CO2 [g] user-defined parameter
m_c5 = 10.18 # mass of cyclopentane [g] (see calculation in Sharepoint > Ternary Sampling (Dow): Ternary Phase Diagram Plan Sheet 2)
m_poly = 40.74 # mass of polyol [g] (see calculation in Sharepoint > Ternary Sampling (Dow): Ternary Phase Diagram Plan Sheet 2)
p_thresh = 0.02 # threshold for pressure [MPa] at which point the algorithm stops searching
rho_thresh = 0.05 # threshold for density [g/mL] at which point the algorithm deems the value equal
m_thresh = 0.5 # threshold for mass [g] at which point the algorithm deems the value equal

def get_p(filepath):
    """Returns pressure as a float given the data filepath."""
    i_p_start = filepath.find('_P') + 2
    i_p_end = filepath.find('_phase_data')
    p_str = filepath[i_p_start:i_p_end]
    if '-' in p_str:
        i_hyphen = p_str.find('-')
        p = int(p_str[:i_hyphen]) + 0.1*int(p_str[i_hyphen+1:])
    else:
        p = int(p_str)
        
    return p

def get_p_bounds(p, p_arr):
    """Returns indices of p_arr for the two pressures that bound the given pressure."""
    i1 = np.where(p>p_arr)[0][-1]
    i2 = np.where(p<=p_arr)[0][0]
    assert i1 == i2-1, 'indices are not adjacent.'
    
    return p_arr[i1], p_arr[i2]


def get_phase_sep(df, m_co2, m_poly, m_c5):
    """"""
    inds_ph_v, inds_ph_l = get_phase_inds(df)
    n = len(inds_ph_v)
    assert n == len(inds_ph_l), 'different number of data points for phase 1 and phase 2.'
    i1 = 0
    above1 = tie_line_diff(df, i1, m_co2, m_poly, m_c5) > 0
    i2 = n-1
    above2 = tie_line_diff(df, i2, m_co2, m_poly, m_c5) > 0
    
    # if it is not the case that i1 has a tie line below the given point and i2 above, cannot interpolate
    assert not above1 and above2, 'cannot interpolate--available tie lines do not straddle given point.'
    while i2-i1 > 1:
        i = int((i2+i1)/2)
        above = tie_line_diff(df, i, m_co2, m_poly, m_c5) > 0
        # lower the tie line above if the middle tie line is above the point
        if above:
            i2 = i
        # raise the tie line below if the middle tie line is below the point
        else:
            i1 = i
    
    d1 = -tie_line_diff(df, i1, m_co2, m_poly, m_c5)
    d2 = tie_line_diff(df, i2, m_co2, m_poly, m_c5)
    a = 1 - d1/(d1+d2)
    b = d1/(d1+d2)
    rho_co2_v = a*df['rhoCO2 [g/mL]'].iloc[inds_ph_v[i1]] + b*df['rhoCO2 [g/mL]'].iloc[inds_ph_v[i2]]
    rho_co2_l = a*df['rhoCO2 [g/mL]'].iloc[inds_ph_l[i1]] + b*df['rhoCO2 [g/mL]'].iloc[inds_ph_l[i2]]
    rho_poly_v = a*df['rhoPoly [g/mL]'].iloc[inds_ph_v[i1]] + b*df['rhoPoly [g/mL]'].iloc[inds_ph_v[i2]]
    rho_poly_l = a*df['rhoPoly [g/mL]'].iloc[inds_ph_l[i1]] + b*df['rhoPoly [g/mL]'].iloc[inds_ph_l[i2]]
    rho_c5_v = a*df['rhoC5 [g/mL]'].iloc[inds_ph_v[i1]] + b*df['rhoC5 [g/mL]'].iloc[inds_ph_v[i2]]
    rho_c5_l = a*df['rhoC5 [g/mL]'].iloc[inds_ph_l[i1]] + b*df['rhoC5 [g/mL]'].iloc[inds_ph_l[i2]]
    w_co2_v = a*df['wCO2'].iloc[inds_ph_v[i1]] + b*df['wCO2'].iloc[inds_ph_v[i2]]
    w_co2_l = a*df['wCO2'].iloc[inds_ph_l[i1]] + b*df['wCO2'].iloc[inds_ph_l[i2]]
    w_poly_v = a*df['wPoly'].iloc[inds_ph_v[i1]] + b*df['wPoly'].iloc[inds_ph_v[i2]]
    w_poly_l = a*df['wPoly'].iloc[inds_ph_l[i1]] + b*df['wPoly'].iloc[inds_ph_l[i2]]
    w_c5_v = a*df['wC5'].iloc[inds_ph_v[i1]] + b*df['wC5'].iloc[inds_ph_v[i2]]
    w_c5_l = a*df['wC5'].iloc[inds_ph_l[i1]] + b*df['wC5'].iloc[inds_ph_l[i2]]
    
    # normalize interpolated weight fractions
    w_co2_v, w_poly_v, w_c5_v = get_wt_frac(w_co2_v, w_poly_v, w_c5_v)
    w_co2_l, w_poly_l, w_c5_l = get_wt_frac(w_co2_l, w_poly_l, w_c5_l)
    
    return w_co2_v, w_poly_l, w_c5_v, w_co2_l, w_poly_v, w_c5_l, \
    rho_co2_v, rho_co2_l, rho_poly_v, rho_poly_l, rho_c5_v, rho_c5_l


def get_phase_inds(df, coex='co2-poly', ph1='co2', ph2='poly'):
    """Returns indices for the two phases."""
    inds = np.where(df['Coexistence'] == coex)[0]
    ph_list = df['Phase'].to_numpy()[inds]
    inds1 = np.array([i for i in inds if df['Phase'].iloc[i]==ph1])
    inds2 = np.array([i for i in inds if df['Phase'].iloc[i]==ph2])  
    
    return inds1, inds2
    
def get_wt_frac(m_co2, m_poly, m_c5):
    """Returns weight fractions given the masses."""
    m = m_co2 + m_poly + m_c5
    w_co2 = m_co2/m
    w_poly = m_poly/m
    w_c5 = m_c5/m
    
    return w_co2, w_poly, w_c5


def tie_line_diff(df, i, m_co2, m_poly, m_c5, coex='co2-poly'):
    """Determines if tie line is above or below the given composition in a ternary plot with CO2 up and polyol to left."""
    # get weight fractions of given recipe in the Parr reactor
    w_co2, w_poly, w_c5 = get_wt_frac(m_co2, m_poly, m_c5)
    
    # get weight fractions on the two sides of the tie line
    inds_ph_v, inds_ph_l = get_phase_inds(df)
    i1 = inds_ph_v[i]
    i2 = inds_ph_l[i]
    w_co2_1 = df['wCO2'].iloc[i1]
    w_co2_2 = df['wCO2'].iloc[i2]
    w_c5_1 = df['wC5'].iloc[i1]
    w_c5_2 = df['wC5'].iloc[i2]
    
    # compute w_co2 along this tie line
    w_c5_pred = (w_c5_2 - w_c5_1)/(w_co2_2 - w_co2_1)*(w_co2 - w_co2_1) + w_c5_1
    
    return w_c5_pred - w_c5
    
    
    
def get_phase_sep_interp_p(p, df_dict, m_co2, m_poly, m_c5):
    """"""
    p_arr = np.sort(np.array(list(df_dict.keys())))
    p1, p2 = get_p_bounds(p, p_arr)
    df1 = df_dict[p1]
    df2 = df_dict[p2]
    
    w_co2_v_1, w_poly_l_1, w_c5_v_1, w_co2_l_1, w_poly_v_1, w_c5_l_1, \
    rho_co2_v_1, rho_co2_l_1, rho_poly_v_1, rho_poly_l_1, rho_c5_v_1, rho_c5_l_1 = get_phase_sep(df1, m_co2, m_poly, m_c5)
    w_co2_v_2, w_poly_l_2, w_c5_v_2, w_co2_l_2, w_poly_v_2, w_c5_l_2, \
    rho_co2_v_2, rho_co2_l_2, rho_poly_v_2, rho_poly_l_2, rho_c5_v_2, rho_c5_l_2 = get_phase_sep(df2, m_co2, m_poly, m_c5)
    
    # coefficients for interpolating between pressures
    a1 = (1 - (p-p1)/(p2-p1))
    a2 = (p-p1)/(p2-p1)
      
    # linearly interpolate weight fractions
    w_co2_v = a1*w_co2_v_1 + a2*w_co2_v_2
    w_co2_l = a1*w_co2_l_1 + a2*w_co2_l_2
    w_poly_v = a1*w_poly_v_1 + a2*w_poly_v_2
    w_poly_l = a1*w_poly_l_1 + a2*w_poly_l_2
    w_c5_v = a1*w_c5_v_1 + a2*w_c5_v_2
    w_c5_l = a1*w_c5_l_1 + a2*w_c5_l_2
    # normalize interpolated weight fractions
    w_co2_v, w_poly_v, w_c5_v = get_wt_frac(w_co2_v, w_poly_v, w_c5_v)
    w_co2_l, w_poly_l, w_c5_l = get_wt_frac(w_co2_l, w_poly_l, w_c5_l)
    
    # linearly interpolate densities
    rho_co2_v = a1*rho_co2_v_1 + a2*rho_co2_v_2
    rho_co2_l = a1*rho_co2_l_1 + a2*rho_co2_l_2
    rho_poly_v = a1*rho_poly_v_1 + a2*rho_poly_v_2
    rho_poly_l = a1*rho_poly_l_1 + a2*rho_poly_l_2
    rho_c5_v = a1*rho_c5_v_1 + a2*rho_c5_v_2
    rho_c5_l = a1*rho_c5_l_1 + a2*rho_c5_l_2
    
    return w_co2_v, w_poly_l, w_c5_v, w_co2_l, w_poly_v, w_c5_l, \
    rho_co2_v, rho_co2_l, rho_poly_v, rho_poly_l, rho_c5_v, rho_c5_l

    
def is_p_high(p, df_dict, m_co2, m_poly, m_c5, V):
    """Indicates whether guess for pressure is too high for the given conditions."""
    V_pred = pred_V(p, df_dict, m_co2, m_poly, m_c5)
    is_p_high = V_pred < V       
    
    return is_p_high


def est_p(df_dict, m_co2, m_poly, m_c5, V, p_thresh, ct_max=100):
    """Estimates pressure with binary search."""
    p_arr = np.sort(np.array(list(df_dict.keys())))
    p_lo = np.min(p_arr)
    p_hi = np.max(p_arr)
    ct = 0
    ct_max = 100
    while (p_hi - p_lo) > p_thresh and ct < ct_max:
        # bisect the pressure limits
        p = (p_hi + p_lo)/2
        is_high = is_p_high(p, df_dict_red, m_co2, m_poly, m_c5, V)
        print("is p high? " + str(is_high))
        if is_high:
            p_hi = p
        else:
            p_lo = p

        ct += 1

    return p



def pred_V(p, df_dict, m_co2, m_poly, m_c5):
    w_co2_v, w_poly_l, w_c5_v, w_co2_l, w_poly_v, w_c5_l, \
    rho_co2_v, rho_co2_l, rho_poly_v, rho_poly_l, rho_c5_v, rho_c5_l = get_phase_sep_interp_p(p, df_dict, 
                                                                                              m_co2, m_poly, m_c5)
    
    # compute mass in vapor phase using lever rule
    m_v = (m_co2/w_co2_l - m_c5/w_c5_l)/((w_co2_v/w_co2_l) - (w_c5_v/w_c5_l))
    # compute mass in liquid phase 
    m = m_co2 + m_poly + m_c5
    m_l = m - m_v
#     print((m_co2/w_co2_v - m_c5/w_c5_v)/((w_co2_l/w_co2_v) - (w_c5_l/w_c5_v))) # check that eqn is consistent w/ lever rule
    # sum to compute volume
    rho_v = rho_co2_v + rho_poly_v + rho_c5_v
    rho_l = rho_co2_l + rho_poly_l + rho_c5_l
    V_pred = m_v/rho_v + m_l/rho_l
    
    return V_pred

def is_m_co2_high(m_co2, df_dict, p, m_poly, m_c5, V):
    """Indicates whether guess for mass of CO2 to add is too high for the given conditions."""
    V_pred = pred_V(p, df_dict, m_co2, m_poly, m_c5)
    is_m_co2_high = V_pred > V
    
    return is_m_co2_high


def est_m_co2(df_dict, p, m_poly, m_c5, V, m_thresh, ct_max=100, m_co2_min=0, m_co2_max=1000):
    """Estimates pressure with binary search."""
    ct = 0
    ct_max = 100
    m_co2_lo = m_co2_min
    m_co2_hi = m_co2_max
    while (m_co2_hi - m_co2_lo) > m_thresh and ct < ct_max:
        # bisect the CO2 mass limits
        m_co2 = (m_co2_hi + m_co2_lo)/2
        is_high = is_m_co2_high(m_co2, df_dict, p, m_poly, m_c5, V)
        print("is m_co2 high? " + str(is_high))
        if is_high:
            m_co2_hi = m_co2
        else:
            m_co2_lo = m_co2

        ct += 1

    return m_co2

## Load Data

Load the data files for the ternary system properties.

In [151]:
# Generate list of data files to use--the more the better the accuracy
data_filepath_list = glob.glob(os.path.join(data_folder + data_file_format))

# Create dictionary of dataframes to access data
df_dict = {}
for data_filepath in data_filepath_list:
    p = get_p(data_filepath)
    df = pd.read_csv(data_filepath)
    df_dict[p] = df
    
df_dict_red = {}
df_dict_red[3] = df_dict[3]
df_dict_red[7] = df_dict[7]

In [152]:
p = est_p(df_dict_red, m_co2, m_poly, m_c5, V, p_thresh)
print(p)
print(get_phase_sep_interp_p(p, df_dict_red, m_co2, m_poly, m_c5))

# compare interpolated phase separation concentrations to those predicted by Huikuan's model at 5.9 MPa
df = df_dict[5.9]
print(get_phase_sep(df, m_co2, m_poly, m_c5))

is p high? False
is p high? True
is p high? False
is p high? False
is p high? False
is p high? True
is p high? True
is p high? False
5.890625
(0.981779049473682, 0.6491992464398648, 0.01822095052631809, 0.244179180972949, 2.2273329971115135e-27, 0.10662157258718608, 0.16304881425582515, 0.25951446671312867, 4.601269627073204e-28, 0.6916457046314671, 0.0029686240915748627, 0.11352221502406529)
(0.9833125380352359, 0.6486179056325126, 0.016687461964764155, 0.23980772275338103, 3.810379601358352e-30, 0.11157437161410638, 0.14721834248436982, 0.25500233595377564, 5.704169250246708e-31, 0.689765732165775, 0.002498565812064999, 0.1186221588027947)


In [153]:
p = 5.9 # MPa
m_co2_pred = est_m_co2(df_dict, p, m_poly, m_c5, V, m_thresh)
print(m_co2_pred)
print(m_co2)

is m_co2 high? True
is m_co2 high? True
is m_co2 high? False
is m_co2 high? True
is m_co2 high? False
is m_co2 high? False
is m_co2 high? False
is m_co2 high? True
is m_co2 high? False
is m_co2 high? False
is m_co2 high? False
183.10546875
202


In [36]:
# TODO Fix the discrepancy above^

TypeError: only integer scalar arrays can be converted to a scalar index